In [1]:
import pandas as pd
import os
from PIL import Image

In [2]:
import torch
import timm
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from torchvision import transforms
from tqdm import tqdm
from sklearn.svm import LinearSVR
from scipy.stats import pearsonr
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

In [3]:
# === Set paths ===
csv_path = "C:\\Users\\anush\\Downloads\\BMI\\Data\\data.csv" 
image_folder = "C:\\Users\\anush\\Downloads\\BMI\\Data\\Images" 

# === Step 1: Load metadata ===
df = pd.read_csv(csv_path)

# === Step 2: Define image loader ===
def load_image(file_name):
    img_path = os.path.join(image_folder, file_name)
    try:
        return Image.open(img_path).convert('RGB')  # Convert to RGB for consistency
    except Exception as e:
        return None

# === Step 3: Load images ===
df['image'] = df['name'].apply(load_image)

# === Step 4: Drop rows where image failed to load ===
df = df[df['image'].notnull()].reset_index(drop=True)

# === Step 5: Keep only needed columns ===
df = df[['image', 'bmi', 'gender']]

print(f"Loaded {len(df)} valid samples.")
print(df.head())

Loaded 3962 valid samples.
                                               image        bmi  gender
0  <PIL.Image.Image image mode=RGB size=164x176 a...  34.207396    Male
1  <PIL.Image.Image image mode=RGB size=150x173 a...  26.453720    Male
2  <PIL.Image.Image image mode=RGB size=226x251 a...  34.967561  Female
3  <PIL.Image.Image image mode=RGB size=97x126 at...  22.044766  Female
4  <PIL.Image.Image image mode=RGB size=87x115 at...  25.845588  Female


In [4]:
# Step 3: Split the data
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 7)

In [5]:
# Step 4: Define image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

In [9]:
# Step 5: Load pre-trained ViT (no classifier head)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model('deit_tiny_patch16_224', pretrained=True)
model.head = torch.nn.Identity()  # Remove classification layer
model.to(device)
model.eval()

model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anush\.cache\huggingface\hub\models--timm--deit_tiny_patch16_224.fb_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()


In [10]:
# Step 6: Extract features (plain for loop, no class)
def extract_features(images):
    features = []

    for img in tqdm(images):
        if img is None:
            continue
        with torch.no_grad():
            img_tensor = transform(img).unsqueeze(0).to(device)
            output = model(img_tensor)
            features.append(output.squeeze().cpu().numpy())
    
    return np.vstack(features)

In [12]:
# Step 7: Extract train/test features and targets
X_train = extract_features(train_df["image"])
y_train = train_df["bmi"].values

100%|██████████| 3169/3169 [01:44<00:00, 30.45it/s]


In [13]:
X_test = extract_features(test_df["image"])
y_test = test_df["bmi"].values

100%|██████████| 793/793 [00:36<00:00, 21.44it/s]


In [14]:
svr = LinearSVR(C = 1.0, max_iter = 10000)
svr.fit(X_train, y_train)

LinearSVR(max_iter=10000)

In [15]:
y_pred = svr.predict(X_test)

In [16]:
gender_test = test_df['gender'].values

# Overall Pearson r
r_all, _ = pearsonr(y_test, y_pred)

# By gender
mask_male = (gender_test == 'Male')
mask_female = (gender_test == 'Female')

r_male, _ = pearsonr(y_test[mask_male], y_pred[mask_male])
r_female, _ = pearsonr(y_test[mask_female], y_pred[mask_female])

# Print results
print("Pearson r correlations:")
print(f"  Male:    {r_male:.2f}")
print(f"  Female:  {r_female:.2f}")
print(f"  Overall: {r_all:.2f}")

Pearson r correlations:
  Male:    0.37
  Female:  0.25
  Overall: 0.32


In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
svr2 = LinearSVR(C = 1.0, max_iter = 10000)
svr2.fit(X_train_scaled, y_train)

LinearSVR(max_iter=10000)

In [19]:
y_pred = svr.predict(X_test_scaled)

In [20]:
# Overall Pearson r
r_all, _ = pearsonr(y_test, y_pred)

# By gender
mask_male = (gender_test == 'Male')
mask_female = (gender_test == 'Female')

r_male, _ = pearsonr(y_test[mask_male], y_pred[mask_male])
r_female, _ = pearsonr(y_test[mask_female], y_pred[mask_female])

# Print results
print("Pearson r correlations:")
print(f"  Male:    {r_male:.2f}")
print(f"  Female:  {r_female:.2f}")
print(f"  Overall: {r_all:.2f}")

Pearson r correlations:
  Male:    0.33
  Female:  0.17
  Overall: 0.27


In [21]:
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.5, 1.0]
}

grid = GridSearchCV(LinearSVR(max_iter=10000), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)

c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

Best params: {'C': 10, 'epsilon': 1.0}


c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [22]:
y_pred = grid.predict(X_test)

In [23]:
# Overall Pearson r
r_all, _ = pearsonr(y_test, y_pred)

# By gender
mask_male = (gender_test == 'Male')
mask_female = (gender_test == 'Female')

r_male, _ = pearsonr(y_test[mask_male], y_pred[mask_male])
r_female, _ = pearsonr(y_test[mask_female], y_pred[mask_female])

# Print results
print("Pearson r correlations:")
print(f"  Male:    {r_male:.2f}")
print(f"  Female:  {r_female:.2f}")
print(f"  Overall: {r_all:.2f}")

Pearson r correlations:
  Male:    0.40
  Female:  0.27
  Overall: 0.35


In [24]:
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}


In [25]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1,
    random_state= 7
)

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=20,                
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state= 7
)

random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraint...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=-1,
                                          num_parallel_tree=None, ...),
                   n_iter=20,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=7, scoring='neg_mean_squared_error', verbose=1)

In [26]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

In [27]:
# Overall Pearson r
r_all, _ = pearsonr(y_test, y_pred)

# By gender
mask_male = (gender_test == 'Male')
mask_female = (gender_test == 'Female')

r_male, _ = pearsonr(y_test[mask_male], y_pred[mask_male])
r_female, _ = pearsonr(y_test[mask_female], y_pred[mask_female])

# Print results
print("Pearson r correlations:")
print(f"  Male:    {r_male:.2f}")
print(f"  Female:  {r_female:.2f}")
print(f"  Overall: {r_all:.2f}")

Pearson r correlations:
  Male:    0.34
  Female:  0.23
  Overall: 0.30


In [28]:
# === 1. Define parameter grid ===
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}

# === 2. Initialize Ridge model and GridSearchCV ===
ridge = Ridge()
grid = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # or 'r2' if you prefer
    cv=5,
    verbose=1,
    n_jobs=-1
)

# === 3. Fit the grid search ===
grid.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [29]:
# === 4. Get best model and predict ===
best_ridge = grid.best_estimator_
y_pred = best_ridge.predict(X_test)

In [30]:
# Overall Pearson r
r_all, _ = pearsonr(y_test, y_pred)

# By gender
mask_male = (gender_test == 'Male')
mask_female = (gender_test == 'Female')

r_male, _ = pearsonr(y_test[mask_male], y_pred[mask_male])
r_female, _ = pearsonr(y_test[mask_female], y_pred[mask_female])

# Print results
print("Pearson r correlations:")
print(f"  Male:    {r_male:.2f}")
print(f"  Female:  {r_female:.2f}")
print(f"  Overall: {r_all:.2f}")

Pearson r correlations:
  Male:    0.41
  Female:  0.28
  Overall: 0.36


In [31]:
# === 1. Define parameter grid ===
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}

# === 2. Initialize Ridge model and GridSearchCV ===
ridge2 = Ridge()
grid2 = GridSearchCV(
    estimator=ridge2,
    param_grid=param_grid,
    scoring='r2',  
    cv=5,
    verbose=1,
    n_jobs=-1
)

# === 3. Fit the grid search ===
grid2.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}, scoring='r2',
             verbose=1)

In [32]:
# === 4. Get best model and predict ===
best_ridge2 = grid2.best_estimator_
y_pred = best_ridge2.predict(X_test)

In [33]:
# Overall Pearson r
r_all, _ = pearsonr(y_test, y_pred)

# By gender
mask_male = (gender_test == 'Male')
mask_female = (gender_test == 'Female')

r_male, _ = pearsonr(y_test[mask_male], y_pred[mask_male])
r_female, _ = pearsonr(y_test[mask_female], y_pred[mask_female])

# Print results
print("Pearson r correlations:")
print(f"  Male:    {r_male:.2f}")
print(f"  Female:  {r_female:.2f}")
print(f"  Overall: {r_all:.2f}")

Pearson r correlations:
  Male:    0.41
  Female:  0.28
  Overall: 0.36
